# CIAN.ru

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [3]:
page = 1

### Для ЦАО находим главную ссылку и в ней находим **&p=1&**, это и есть переход по страницам

In [4]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [5]:
district

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

### Гуляем по страницам и качаем с них все ссылки на наши квартирки

In [6]:
links = []
for page in range(1, 30):
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')

    flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

    for link in flat_urls:
        if link.isdigit():
            links.append(link)

In [7]:
links[0]

'151051665'

In [8]:
len(links)

812

Итого вытащили 812 квартирок по ЦАО

### Попробуем перейти на первую

In [9]:
flat_url = 'http://www.cian.ru/sale/flat/' + str(links[0]) + '/'
#flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

In [10]:
flatStats = {'District':0}

# Цена

In [11]:
price = flat_page.find('div', attrs={'class':'object_descr_price'})
price = re.split('<div>|за м<sup>2</sup>|\n|руб|\W', str(price))

In [12]:
print(price)

['', 'div', 'class', '', 'object_descr_price', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '18', '356', '000', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'div', '']


In [13]:
[i for i in price if i.isdigit()][-3:]

['18', '356', '000']

In [14]:
"".join([i for i in price if i.isdigit()][-3:])

'18356000'

In [15]:
price = "".join([i for i in price if i.isdigit()][-3:])

In [16]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [17]:
getPrice(flat_page)

18356000

In [18]:
flatStats['Price'] = getPrice(flat_page)

# Координаты

In [19]:
flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]

<a href="http://map.cian.ru/?deal_type=2&amp;flats=yes&amp;center=55.770208%2C37.638036&amp;room0=1&amp;room1=1" rel="nofollow" target="_blank"><i></i>Посмотреть предложения рядом</a>

In [20]:
coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
coords = re.split('&amp|center=|%2C', str(coords))
coords_list = []
for item in coords:
    if item[0].isdigit():
        coords_list.append(item)
lat = float(coords_list[0])
lon = float(coords_list[1])

In [21]:
lat, lon

(55.770208, 37.638036)

In [22]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

In [23]:
getCoords(flat_page)

(55.770208, 37.638036)

In [24]:
flatStats['lat'], flatStats['lon'] = getCoords(flat_page)

# Число комнат

In [25]:
rooms = flat_page.find('div', attrs={'class':'object_descr_title'})

In [26]:
rooms

<div class="object_descr_title" itemprop="name">
                
    
        1-комн. кв. 
    

                
                    
                        в <a href="/zhiloy-kompleks-klubnyy-dom-na-sretenke-moskva-7099/" target="_blank">ЖК «Клубный дом на Сретенке»</a>
</div>

In [27]:
rooms = html_stripper(rooms)

In [28]:
rooms

'\n                \n    \n        1-комн. кв.\xa0\n    \n\n                \n                    \n                        в\xa0ЖК «Клубный дом на Сретенке»\n'

In [29]:
room_number = ''
for i in re.split('-|\n', rooms):
    if 'комн' in i:
        break
    else:
        room_number += i
room_number = "".join(room_number.split())

In [30]:
room_number

'1'

In [31]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number

In [32]:
getRoom(flat_page)

'1'

In [33]:
flatStats['rooms'] = getRoom(flat_page)

# Общая площадь квартиры


In [34]:
totsp = flat_page.find('div', attrs={'class':'clearfix'})

In [35]:
totsp

<div class="clearfix" style="position: relative">
<table class="object_descr_props flat sale" style="float:left">
<tr>
<th style="padding-top:5px;"> # Общая информация:</th>
</tr>
<tr>
<th>Этаж:</th>
<td>
                3 / 8
                
            </td>
</tr>
<tr>
<th>Тип дома:</th>
<td>
            новостройка, 
            монолитный
        </td>
</tr>
<tr>
<th>Тип продажи:</th>
<td>свободная</td>
</tr>
<tr>
<th>Общая площадь:</th>
<td>
<i class="object_descr_details_color"></i>48,8 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь комнат:</th>
<td>
<i class="object_descr_details_color"></i><!--
-->–
        </td>
</tr>
<tr>
<th>Жилая площадь:</th>
<td>
<i class="object_descr_details_color" style="background-color:#90C090"></i>18 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь кухни:</th>
<td>
<i class="object_descr_details_color" style="background-color:#C09090"></i>10 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Раздельных санузлов:</th>
<td><i class="object_descr_details_color" style="backgrou

In [36]:
totsp = html_stripper(totsp)

In [37]:
totsp

"\n\n\n # Общая информация:\n\n\nЭтаж:\n\n                3\xa0/\xa08\n                \n            \n\n\nТип дома:\n\n            новостройка, \n            монолитный\n        \n\n\nТип продажи:\nсвободная\n\n\nОбщая площадь:\n\n48,8\xa0м2\n\n\n\nПлощадь комнат:\n\n–\n        \n\n\nЖилая площадь:\n\n18\xa0м2\n\n\n\nПлощадь кухни:\n\n10\xa0м2\n\n\n\nРаздельных санузлов:\n1\n            \n\n\nСовмещенных санузлов:\n1\n            \n\n\nБалкон:\n1 балк. + 1 лодж.\n        \n\n\nЛифт:\n4 пасс. + 4 груз.\n\n\nВид из окна:\nдвор и улица\n\n\nСдача ГК:\n3 кв. 2017 года\n\n\n\n\n\n<!--Категория: -->\n\n\n\n\n<!--\nif (typeof(pr) == 'undefined') { var pr = Math.floor(Math.random() * 4294967295) + 1; }\nif (typeof(document.referrer) != 'undefined') {\n  if (typeof(afReferrer) == 'undefined') {\n    afReferrer = encodeURIComponent(document.referrer);\n  }\n} else {\n  afReferrer = '';\n}\nvar addate = new Date();\n\n\nvar dl = encodeURIComponent(document.location);\nvar pr1 = Math.floor(Math.r

In [38]:
totsp_number = ''
for i in re.split('-|\n', totsp):
    if 'Площадь комнат' in i:
        break
    else:
        totsp_number += i
totsp_number = "".join(totsp_number.split())

totsp_m2 = totsp_number.split(':')[-1].split('м')[0]

In [39]:
totsp_m2

'48,8'

In [40]:
def gettotsp(flat_page):
    totsp = flat_page.find('div', attrs={'class':'clearfix'})
    totsp = html_stripper(totsp)
    totsp_number = ''
    for i in re.split('-|\n', totsp):
        if 'Площадь комнат' in i:
            break
        else:
            totsp_number += i
    totsp_number = "".join(totsp_number.split())
    totsp_m2 = totsp_number.split(':')[-1].split('м')[0]
    return totsp_m2

In [41]:
gettotsp(flat_page)

'48,8'

In [42]:
flatStats['totsp'] = gettotsp(flat_page)

# Жилое пространство

In [43]:
livesp = flat_page.find('div', attrs={'class':'clearfix'})

In [44]:
livesp

<div class="clearfix" style="position: relative">
<table class="object_descr_props flat sale" style="float:left">
<tr>
<th style="padding-top:5px;"> # Общая информация:</th>
</tr>
<tr>
<th>Этаж:</th>
<td>
                3 / 8
                
            </td>
</tr>
<tr>
<th>Тип дома:</th>
<td>
            новостройка, 
            монолитный
        </td>
</tr>
<tr>
<th>Тип продажи:</th>
<td>свободная</td>
</tr>
<tr>
<th>Общая площадь:</th>
<td>
<i class="object_descr_details_color"></i>48,8 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь комнат:</th>
<td>
<i class="object_descr_details_color"></i><!--
-->–
        </td>
</tr>
<tr>
<th>Жилая площадь:</th>
<td>
<i class="object_descr_details_color" style="background-color:#90C090"></i>18 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Площадь кухни:</th>
<td>
<i class="object_descr_details_color" style="background-color:#C09090"></i>10 м<sup>2</sup>
</td>
</tr>
<tr>
<th>Раздельных санузлов:</th>
<td><i class="object_descr_details_color" style="backgrou

In [45]:
livesp = html_stripper(livesp)

In [46]:
livesp

"\n\n\n # Общая информация:\n\n\nЭтаж:\n\n                3\xa0/\xa08\n                \n            \n\n\nТип дома:\n\n            новостройка, \n            монолитный\n        \n\n\nТип продажи:\nсвободная\n\n\nОбщая площадь:\n\n48,8\xa0м2\n\n\n\nПлощадь комнат:\n\n–\n        \n\n\nЖилая площадь:\n\n18\xa0м2\n\n\n\nПлощадь кухни:\n\n10\xa0м2\n\n\n\nРаздельных санузлов:\n1\n            \n\n\nСовмещенных санузлов:\n1\n            \n\n\nБалкон:\n1 балк. + 1 лодж.\n        \n\n\nЛифт:\n4 пасс. + 4 груз.\n\n\nВид из окна:\nдвор и улица\n\n\nСдача ГК:\n3 кв. 2017 года\n\n\n\n\n\n<!--Категория: -->\n\n\n\n\n<!--\nif (typeof(pr) == 'undefined') { var pr = Math.floor(Math.random() * 4294967295) + 1; }\nif (typeof(document.referrer) != 'undefined') {\n  if (typeof(afReferrer) == 'undefined') {\n    afReferrer = encodeURIComponent(document.referrer);\n  }\n} else {\n  afReferrer = '';\n}\nvar addate = new Date();\n\n\nvar dl = encodeURIComponent(document.location);\nvar pr1 = Math.floor(Math.r

In [47]:
livesp_number = ''
for i in re.split('-|\n', livesp):
    if 'Площадь кухни' in i:
        break
    else:
        livesp_number += i
livesp_number = "".join(livesp_number.split())

livesp_m2 = livesp_number.split(':')[-1].split('м')[0]

In [48]:
def getlivesp(flat_page):
    livesp = flat_page.find('div', attrs={'class':'clearfix'})
    livesp = html_stripper(livesp)
    livesp_number = ''
    for i in re.split('-|\n', livesp):
        if 'Площадь кухни' in i:
            break
        else:
            livesp_number += i
    livesp_number = "".join(livesp_number.split())
    livesp_m2 = livesp_number.split(':')[-1].split('м')[0]
    return livesp_m2

In [49]:
getlivesp(flat_page)

'18'

In [50]:
flatStats['livesp'] = getlivesp(flat_page)

# Площадь кухни

In [51]:
kitsp_number = ''
for i in re.split('-|\n', livesp):
    if 'Санузел' in i:
        break
    else:
        kitsp_number += i
kitsp_number = "".join(kitsp_number.split())

kitsp_m2 = kitsp_number.split(':')[-1].split('м')[0]

In [52]:
def getkitsp(flat_page):
    kitsp = flat_page.find('div', attrs={'class':'clearfix'})
    kitsp = html_stripper(kitsp)
    kitsp_number = ''
    for i in re.split('-|\n', kitsp):
        if 'Санузел' in i:
            break
        else:
            kitsp_number += i
    kitsp_number = "".join(kitsp_number.split())
    kitsp_m2 = kitsp_number.split(':')[-1].split('м')[0]
    return kitsp_m2 

In [53]:
getkitsp(flat_page)

"><!if(typeof(pr)=='undefined'){varpr=Math.floor(Math.random()*4294967295)+1;}if(typeof(document.referrer)!='undefined'){if(typeof(afReferrer)=='undefined'){afReferrer=encodeURIComponent(document.referrer);}}else{afReferrer='';}varaddate=newDate();vardl=encodeURIComponent(document.location);varpr1=Math.floor(Math.random()*4294967295)+1;(function(){varelem=document.getElementById('ad_mortgage_card_exp_1');elem.setAttribute('id','ad_mortgage_card_exp_1'+pr1);elem.innerHTML='';elem.innerHTML+='';})();AdFox_getCodeScript(1,pr1,'//ads.adfox.ru/202100/prepareCode?p1=bvkoc&amp;p2=fkib&amp;pct=a&amp;pfc=a&amp;pfb=a&amp;plp=a&amp;pli=a&amp;pop=a&amp;pr='+pr+'&amp;pt=b&amp;pd='+addate.getDate()+'&amp;pw='+addate.getDay()+'&amp;pv='+addate.getHours()+'&amp;prr='+afReferrer+'&amp;puid17=true&amp;puid16=false&amp;puid10=no_agent&amp;puid7=2323&amp;puid6=15001_20000&amp;puid5=obl_id_1&amp;puid4=offer_type_1&amp;puid3=sale_type_first&amp;puid2=offer_flat&amp;puid1=deal_type_sale&amp;puid9=121&amp;pui

In [54]:
flatStats['kitsp'] = getkitsp(flat_page)

# Тип дома

In [59]:
#brick = flat_page.find('div', attrs={'class':'bti__col'})
#brick = html_stripper(brick)

In [62]:
livesp

"\n\n\n # Общая информация:\n\n\nЭтаж:\n\n                3\xa0/\xa08\n                \n            \n\n\nТип дома:\n\n            новостройка, \n            монолитный\n        \n\n\nТип продажи:\nсвободная\n\n\nОбщая площадь:\n\n48,8\xa0м2\n\n\n\nПлощадь комнат:\n\n–\n        \n\n\nЖилая площадь:\n\n18\xa0м2\n\n\n\nПлощадь кухни:\n\n10\xa0м2\n\n\n\nРаздельных санузлов:\n1\n            \n\n\nСовмещенных санузлов:\n1\n            \n\n\nБалкон:\n1 балк. + 1 лодж.\n        \n\n\nЛифт:\n4 пасс. + 4 груз.\n\n\nВид из окна:\nдвор и улица\n\n\nСдача ГК:\n3 кв. 2017 года\n\n\n\n\n\n<!--Категория: -->\n\n\n\n\n<!--\nif (typeof(pr) == 'undefined') { var pr = Math.floor(Math.random() * 4294967295) + 1; }\nif (typeof(document.referrer) != 'undefined') {\n  if (typeof(afReferrer) == 'undefined') {\n    afReferrer = encodeURIComponent(document.referrer);\n  }\n} else {\n  afReferrer = '';\n}\nvar addate = new Date();\n\n\nvar dl = encodeURIComponent(document.location);\nvar pr1 = Math.floor(Math.r

In [69]:
brick_number = ''
for i in re.split('-|\n', livesp):
    if 'Тип продажи' in i:
        break
    else:
        brick_number += i
brick_number = "".join(brick_number.split()).split(":")[-1]

In [70]:
brick_number

'новостройка,монолитный'

In [72]:
def getbrick(flat_page):
    brick = flat_page.find('div', attrs={'class':'clearfix'})
    brick = html_stripper(brick)
    brick_number = ''
    for i in re.split('-|\n', brick):
        if 'Тип продажи' in i:
            break
        else:
            brick_number += i
    brick_number = "".join(brick_number.split()).split(":")[-1]
    return brick_number 

In [73]:
getbrick(flat_page)

'новостройка,монолитный'

In [74]:
flatStats['brick'] = getbrick(flat_page)

# Телефон

In [86]:
tel = flat_page.find('div', attrs={'class':'realtor-card__phone'})
tel = html_stripper(tel)

In [87]:
tel

'\n+7 495 134-61-40\n'

In [95]:
phone_number = ''
for i in re.split('-|\n', tel):
    if '>' in i:
        break
    else:
        phone_number += i
phone_number # = "".join(brick_number.split()).split(":")[-1]

'+7 495 1346140'

In [96]:
def gettel(flat_page):
    tel = flat_page.find('div', attrs={'class':'realtor-card__phone'})
    tel = html_stripper(tel)
    phone_number = ''
    for i in re.split('-|\n', tel):
        if '>' in i:
            break
        else:
            phone_number += i
    return phone_number 

In [98]:
gettel(flat_page)

'+7 495 1346140'

In [99]:
flatStats['tel'] = gettel(flat_page)

# Балкон

In [154]:
livesp

"\n\n\n # Общая информация:\n\n\nЭтаж:\n\n                3\xa0/\xa04\n                \n            \n\n\nТип дома:\n\n            вторичка\n            \n        \n\n\nТип продажи:\nсвободная\n\n\nОбщая площадь:\n\n185\xa0м2\n\n\n\nПлощадь комнат:\n\n–\n        \n\n\nЖилая площадь:\n\n–\n        \n\n\nПлощадь кухни:\n\n–\n        \n\n\nСанузел:\n–\n        \n\n\nБалкон:\n–\n        \n\n\nЛифт:\n1 пасс.\n\n\nВид из окна:\n–\n\n\n\n\n\n<!--Категория: -->\n\n\n\n\n<!--\nif (typeof(pr) == 'undefined') { var pr = Math.floor(Math.random() * 4294967295) + 1; }\nif (typeof(document.referrer) != 'undefined') {\n  if (typeof(afReferrer) == 'undefined') {\n    afReferrer = encodeURIComponent(document.referrer);\n  }\n} else {\n  afReferrer = '';\n}\nvar addate = new Date();\n\n\nvar dl = encodeURIComponent(document.location);\nvar pr1 = Math.floor(Math.random() * 4294967295) + 1;\n\n(function(){\nvar elem = document.getElementById('ad_mortgage_card_exp_1');\nelem.setAttribute('id', 'ad_mortgage

In [108]:
bal_number = ''
for i in re.split('-|\n', livesp):
    if 'Лифт' in i:
        break
    else:
        bal_number += i
bal_number = "".join(bal_number.split())

bal_is = bal_number.split(':')[-1]

In [110]:
if bal_is=='-':
    bal=0
else: bal=1

In [111]:
bal

1

In [118]:
def getbal(flat_page):
    bal = flat_page.find('div', attrs={'class':'clearfix'})
    bal = html_stripper(bal)
    bal_number = ''
    for i in re.split('-|\n', bal):
        if 'Лифт' in i:
            break
        else:
            bal_number += i
        bal_number = "".join(bal_number.split()).split(':')[-1]
        if bal_is=='-':
            bal=0
        else: bal=1
    return bal

In [120]:
flatStats['bal'] = getbal(flat_page)

# Таблица

In [34]:
table = flat_page.find('table', attrs = {'class':'object_descr_props'})

In [35]:
table

<table class="object_descr_props flat sale" style="float:left">\n<tr>\n<th style="padding-top:5px;"> # \u041e\u0431\u0449\u0430\u044f \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f:</th>\n</tr>\n<tr>\n<th>\u042d\u0442\u0430\u0436:</th>\n<td>\n                9\xa0\n                \n            </td>\n</tr>\n<tr>\n<th>\u0422\u0438\u043f \u0434\u043e\u043c\u0430:</th>\n<td>\n            \u0432\u0442\u043e\u0440\u0438\u0447\u043a\u0430, \n            \u043c\u043e\u043d\u043e\u043b\u0438\u0442\u043d\u044b\u0439\n        </td>\n</tr>\n<tr>\n<th>\u0422\u0438\u043f \u043f\u0440\u043e\u0434\u0430\u0436\u0438:</th>\n<td>\u0441\u0432\u043e\u0431\u043e\u0434\u043d\u0430\u044f</td>\n</tr>\n<tr>\n<th>\u041e\u0431\u0449\u0430\u044f \u043f\u043b\u043e\u0449\u0430\u0434\u044c:</th>\n<td>\n<i class="object_descr_details_color"></i>100\xa0\u043c<sup>2</sup>\n</td>\n</tr>\n<tr>\n<th>\u041f\u043b\u043e\u0449\u0430\u0434\u044c \u043a\u043e\u043c\u043d\u0430\u0442:</th>\n<td>\n<i class="objec

In [36]:
print(html_stripper(table))



 # Общая информация:


Этаж:

                9 
                
            


Тип дома:

            вторичка, 
            монолитный
        


Тип продажи:
свободная


Общая площадь:

100 м2



Площадь комнат:

–
        


Жилая площадь:

79 м2



Площадь кухни:

21 м2



Раздельных санузлов:
2
            


Балкон:
1 балк.
        


Лифт:
1 пасс. + 1 груз.


Телефон:
да


Вид из окна:
–




In [37]:
table = html_stripper(table)

In [38]:
print(re.split('Этаж|Тип дома', table)[1])

:

                9 
                
            





In [39]:
flat_url

'http://www.cian.ru/sale/flat/150913161/'